- sections : '정치', '경제', '사회'
- sections = ["pol", "eco","soc"]
- section_ids = ["100", "101","102"]

In [22]:
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime
import warnings
warnings.filterwarnings('ignore')
import re
import pandas as pd
import time
from datetime import datetime
import os

In [23]:
news_df = pd.DataFrame(
    data=None
    , index=None
    , columns=['date','title','link''content']
)

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

중복 제거 후 행 개수:  0


In [24]:
# ConnectionError방지
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    res = requests.get(url, headers = headers, verify=False)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

In [25]:
# 뉴스의 기본 정보 가져오기
def get_top3_news_info():
    news_urls =[]
    
    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=001"
    
    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)
  
    # 해당 분야 상위 뉴스 3개 가져오기

    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    for li in lis3:
        news_url = li.a.attrs.get('href')
        news_urls.append(news_url)
        
    return news_urls

In [26]:
# 상위 3개 뉴스 크롤링
def F_crawling(news_urls) :
    news_titles = []
    news_contents =[]
    news_dates = []

    for url in news_urls:
        news_html = get_soup_obj(url)

        # 뉴스 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None:
            title = news_html.select_one("#content > div.end_ct > div > h2")

        # 뉴스 본문 가져오기
        content = news_html.select("div#dic_area")
        if content == []:
            content = news_html.select("#articeBody")
            
        # 기사 텍스트만 가져오기
        # list합치기
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern=pattern1, repl='', string=str(title))
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')

        news_titles.append(title)
        news_contents.append(content)

        html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
        news_date = html_date.attrs['data-date-time']
        news_dates.append(news_date)
    
    a = pd.DataFrame({'date':news_dates,'title':news_titles,'link':news_urls,'content':news_contents})

    #중복 행 지우기
    a = a.drop_duplicates(keep='first',ignore_index=True)

    return a

In [28]:
# 5번만 크롤링
i = 1
while i < 6 :
    news_urls = get_top3_news_info()
    a = F_crawling(news_urls)    
    news_df = pd.concat([news_df, a], ignore_index=True, keys=['date','title','link''content'])
    news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
    
    day = datetime.now()
    if not os.path.exists('D:\\big16\\final project\\DATA\\REAL_NEWS_DATA\\real_time_news_{}.csv'.format(day.strftime('%Y%m%d'))):
        news_df.to_csv('D:\\big16\\final project\\DATA\\REAL_NEWS_DATA\\real_time_news_{}.csv'.format(day.strftime('%Y%m%d')), index=False, mode='w', encoding='utf-8-sig')
    else:
        news_df.to_csv('D:\\big16\\final project\\DATA\\REAL_NEWS_DATA\\real_time_news_{}.csv'.format(day.strftime('%Y%m%d')), index=False, mode='a', encoding='utf-8-sig', header=False) 

    print(news_df.tail())

    i += 1
    time.sleep(10)

                  date                                        title  \
1  2023-07-06 12:28:04  경기도교육청-용인시, 반도체마이스터고 신설 위해 '맞손'…2026년 개교 목표   
2  2023-07-06 12:28:01            IAEA 사무총장 "오염수 방류, 마지막 한 방울까지 확인"   
3  2023-07-06 12:29:01                주말 남부지방 또 장맛비…비 가시면 다시 열대야·폭염   
4  2023-07-06 12:29:01                             서울국제유아교육전 ‘북적북적’   
5  2023-07-06 12:29:01                 새마을금고 범정부 대응단 구성…"예금자 보호 총력"   

  linkcontent                                               link  \
1         NaN  https://n.news.naver.com/mnews/article/629/000...   
2         NaN  https://n.news.naver.com/mnews/article/047/000...   
3         NaN  https://n.news.naver.com/mnews/article/018/000...   
4         NaN  https://n.news.naver.com/mnews/article/030/000...   
5         NaN  https://n.news.naver.com/mnews/article/422/000...   

                                             content  
1  [\n\n\n\n\n(왼쪽부터) 이상일 용인시장, 이주호 사회부총리 겸 교육부장관,...  
2  [\n그로시 사무총장, 후쿠시마 방문... "모든 문제 해결할 마법은 없어"\n\n...  

KeyboardInterrupt: 

In [ ]:
# 무한루프 크롤링
while True :
    news_urls = get_top3_news_info()
    a = F_crawling(news_urls)
    news_df = pd.concat([news_df, a], ignore_index=True, keys=['date','title','link''content'])
    news_df = news_df.drop_duplicates(keep='first',ignore_index=True)

    day = datetime.now()
    if not os.path.exists('D:\\big16\\final project\\DATA\\REAL_NEWS_DATA\\real_time_news_{}.csv'.format(day.strftime('%Y%m%d'))):
        news_df.to_csv('D:\\big16\\final project\\DATA\\REAL_NEWS_DATA\\real_time_news_{}.csv'.format(day.strftime('%Y%m%d')), index=False, mode='w', encoding='utf-8-sig')
    else:
        news_df.to_csv('D:\\big16\\final project\\DATA\\REAL_NEWS_DATA\\real_time_news_{}.csv'.format(day.strftime('%Y%m%d')), index=False, mode='a', encoding='utf-8-sig', header=False) 

    print(news_df.tail())

    time.sleep(10)

                  date                             title linkcontent  \
0  2023-07-06 10:54:11  동국제강그룹, 창립 69주년…"여러분이 첫 선배" 의미는?         NaN   
1  2023-07-06 10:54:01   [프로필] 공정위 부위원장에 '조사통' 조홍선 조사관리관         NaN   
2  2023-07-06 10:54:01        군사용 드론 ‘하드 킬’  시험 이렇게 이뤄진다         NaN   

                                                link  \
0  https://n.news.naver.com/mnews/article/003/001...   
1  https://n.news.naver.com/mnews/article/277/000...   
2  https://n.news.naver.com/mnews/article/023/000...   

                                             content  
0  [\n\n\n\n\n[서울=뉴시스]6일 서울 수하동 페럼타워에서 열린 동국홀딩스 창...  
1  [\n\n\n\n\n윤석열 대통령이 공정거래위원회 부위원장으로 조홍선 현 공정거래위...  
2  [\n\t\t\t\t\t\t\t\t\t\t현대차그룹 계열사인 현대위아가 국내 최초로...  
                  date                                   title linkcontent  \
1  2023-07-06 10:54:01         [프로필] 공정위 부위원장에 '조사통' 조홍선 조사관리관         NaN   
2  2023-07-06 10:54:01              군사용 드론 ‘하드 킬’  시험 이렇게 이뤄진다         NaN   
3  2023-07-06 10:55:03  현대모비스·LG

KeyboardInterrupt: 

In [ ]:
news_df.tail()

,date,title,linkcontent,link,content
1,2023-07-06 11:17:05,컬리 푸드 페스타 여러 부스 돌아보는 시민들,NaN,https://n.news.naver.com/mnews/article/119/000...,[\n\n\n\n\nⓒ데일리안 홍금표 기자[데일리안 = 홍금표기자] 6일 오전 서울...
2,2023-07-06 11:17:03,"심플한 외관, 그속에 숨겨진 반전공간 [디자인플러스]",NaN,https://n.news.naver.com/mnews/article/016/000...,[\n“재미없는 건물 싫다” 도심·건물 경계 지우기“정릉 소요재·삼성 아치쌓기·영천...
3,2023-07-06 11:18:07,인산인해 이룬 컬리 푸드 페스타,NaN,https://n.news.naver.com/mnews/article/119/000...,[\n\n\n\n\nⓒ데일리안 홍금표 기자[데일리안 = 홍금표기자] 6일 오전 서울...
4,2023-07-06 11:18:05,은행 가계대출 이자 10년만에 최고...‘빚과의 사투’ 길어진다,NaN,https://n.news.naver.com/mnews/article/016/000...,"[\n5월말 금리 5.06%, 0.02%P 증가신규 대출금리는 전반적 감소세은행채 ..."
5,2023-07-06 11:18:05,"우리은행, 메타버스 창작 콘테스트 개최",NaN,https://n.news.naver.com/mnews/article/119/000...,[\n\n\n\n\n우리은행이 우리 메타버스 창작 콘테스트 예선 참가자와 본선 진출...
